# Build the Neural Network  构建神经网络

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")
device

Using cuda device


'cuda'

## Define the Class  定义类
We define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__. Every nn.Module subclass implements the operations on input data in the forward method.

我们通过子类化 nn 来定义我们的神经网络 。模块 ，并在 __init__ 中初始化神经网络层。每 nn.Module 子类实现了 forward 方法中对 input 数据的作。

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

我们创建一个 NeuralNetwork 的实例，将其移动到设备中，并打印其结构。

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


要使用该模型，我们将输入数据传递给它。这将执行模型的 forward 以及一些后台操作 。不要直接调用 model.forward（）！

在输入上调用模型将返回一个二维张量，其中 dim=0 对应于每个类的 10 个原始预测值的每个输出，dim=1 对应于每个输出的单个值。我们通过将预测概率传递给 nn.Softmax 模块。

In [9]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
logits.shape
logits

tensor([[ 0.0058, -0.0839,  0.0674, -0.0426, -0.0876, -0.0096,  0.0083,  0.0311,
          0.0734, -0.0321]], device='cuda:0', grad_fn=<AddmmBackward0>)

果然.logits里面有负值，通过softmax指数化，全部变成正数的0~1之间的概率分布。

In [ ]:
pred_probab = nn.Softmax(dim=1)(logits) # dim=1 表示在类别维度上计算 Softmax
pred_probab

tensor([[0.0988, 0.0954, 0.1052, 0.0965, 0.0952, 0.1025, 0.1012, 0.1014, 0.1042,
         0.0996]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

在 PyTorch 中，nn.Softmax 是一个用于计算 Softmax 函数 的模块。Softmax 函数通常用于多分类任务中，将模型的原始输出（logits）转换为概率分布。

  是指数函数，用于将 logits 转换为正数。
分母是对所有类别的指数值求和，确保输出是一个概率分布（即所有值的和为 1）。
Softmax 的作用是将输入的 logits 转换为每个类别的概率值，便于理解和解释。



In [13]:
y_pred = pred_probab.argmax(1)
# pred_probab是相似度的概率，y_pred就是最大的那个index，但仍是tensor的数据结构
print(f"Predicted class预测的类别: {y_pred.item()}")

Predicted class预测的类别: 2


### 关于softmax


参数说明
- dim：指定在哪个维度上计算 Softmax。通常设置为 dim=1，表示在类别维度上计算（即对每个样本的 logits 计算 Softmax）。
- dtype（可选）：指定输出的数据类型。

注意事项
1. 与 nn.CrossEntropyLoss 的关系：

在分类任务中，通常不需要显式地使用 nn.Softmax，因为 nn.CrossEntropyLoss 内部已经包含了 Softmax 操作。
如果你同时使用了 nn.Softmax 和 nn.CrossEntropyLoss，会导致双重 Softmax，结果会不正确。

2. 数值稳定性：

直接计算 Softmax 可能会导致数值溢出（尤其是当 logits 的值较大时）。PyTorch 的实现已经考虑了数值稳定性问题，但如果你手动实现 Softmax，可以使用 log_softmax 来提高数值稳定性。

In [10]:
import torch
import torch.nn as nn

# 定义 Softmax 层
softmax = nn.Softmax(dim=1)  # dim=1 表示在类别维度上计算 Softmax

# 输入 logits（假设有 3 个样本，每个样本有 5 个类别）
logits = torch.tensor([[2.0, 1.0, 0.1, 0.5, 0.3],
                       [0.5, 2.0, 0.3, 0.2, 0.1],
                       [0.1, 0.2, 0.3, 2.0, 0.4]])

# 计算 Softmax
probabilities = softmax(logits)

print(probabilities)


tensor([[0.5200, 0.1913, 0.0778, 0.1160, 0.0950],
        [0.1297, 0.5812, 0.1062, 0.0961, 0.0869],
        [0.0880, 0.0973, 0.1075, 0.5884, 0.1188]])


### argmax(1)是什么意思

在 PyTorch 中，argmax(dim=1) 是一个常用的操作，用于沿着指定的维度（这里是 dim=1）找到张量中最大值的索引。它通常用于分类任务中，用来确定模型预测的类别。

argmax 的作用
argmax 的全称是 "argument of the maximum"，即返回张量中最大值所在的索引位置。

**argmax就是取得的概率中最大的一个的index，**

## Model Layers  模型层

让我们分解 FashionMNIST 模型中的层。为了说明这一点，我们将取一个包含 3 张大小为 28x28 的图像的示例小批量，看看当我们通过网络传递它时会发生什么。

(0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
从这里可以看到，
784(28x28)---> 512 ----->512---->10

In [14]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten  nn.扁平 化

In [25]:
flatten = nn.Flatten() # 默认值为1
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [23]:
flatten = nn.Flatten(0)
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([2352])


### nn.Linear  nn.线性

In [26]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU


Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

非线性激活是在模型的输入和输出之间创建复杂的映射。它们在线性变换后应用，以引入非线性 ，帮助神经网络学习各种现象。

In this model, we use nn.ReLU between our linear layers, but there’s other activations to introduce non-linearity in your model.

在这个模型中，我们使用 nn.ReLU 的 Alpha 触发器，但还有其他激活可以在模型中引入非线性。

In [27]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0772,  0.4450, -0.1964, -0.1748,  0.1241, -0.1785, -0.0499, -0.5188,
         -0.7385, -0.4252, -0.4887, -0.2668,  0.0503, -0.1330,  0.2156, -0.2457,
          0.3538, -0.2508, -0.0464,  0.1807],
        [-0.1208, -0.0384, -0.1596,  0.1612,  0.5230,  0.1200, -0.0738, -0.4275,
         -0.4815, -0.4379, -0.7345,  0.1661, -0.1340, -0.2579,  0.4045, -0.4077,
          0.2405,  0.1065, -0.3617,  0.1372],
        [-0.1527, -0.1753, -0.0644, -0.4783,  0.0887, -0.0536, -0.0168, -0.0345,
         -0.5114, -0.2443, -0.6038,  0.0185,  0.2780, -0.3482,  0.5325, -0.1480,
          0.3587, -0.2044, -0.2157,  0.1315]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.4450, 0.0000, 0.0000, 0.1241, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0503, 0.0000, 0.2156, 0.0000, 0.3538, 0.0000,
         0.0000, 0.1807],
        [0.0000, 0.0000, 0.0000, 0.1612, 0.5230, 0.1200, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.1661, 0.0000, 0.0000, 0.40

### nn.Sequential  nn.顺序

In [29]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
logits

tensor([[-0.1972, -0.1808, -0.3109, -0.0765, -0.0506, -0.0739, -0.0880,  0.1524,
          0.1210, -0.1520],
        [-0.1323, -0.1009, -0.1739,  0.0079,  0.0453, -0.0859, -0.2262,  0.0022,
         -0.0255, -0.1918],
        [-0.2621,  0.0340, -0.2054,  0.1305, -0.1471, -0.0893, -0.1760,  0.0400,
          0.0732, -0.2025]], grad_fn=<AddmmBackward0>)

### Model Parameters

In [30]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0088, -0.0144,  0.0029,  ..., -0.0025, -0.0235,  0.0056],
        [-0.0182, -0.0199, -0.0036,  ..., -0.0148,  0.0171,  0.0303]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0107, -0.0289], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0050, -0.0214,  0.0145,  ..., -0.0189, -0.0392,  0.0222],
        [ 0.0191,  0.0116,  0.0370,  ..., -0.0273,  0.0430,  0.0432]],
       device='cuda:0', grad_fn=<Sl